In [1]:
x_train_file_path = 'train_x.csv/train_x.csv'
x_test_file_path = 'test_x.csv/test_x.csv'
y_train_file_path = 'train_y.csv'
sample_sub = 'sample_submission.csv'
cx_text = 'CleanedXtest.csv'

In [2]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from keras.layers import Dense, SimpleRNN
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
# get the corrisponding labels
y_train = pd.read_csv(y_train_file_path, low_memory=False)
x_train = pd.read_csv(x_train_file_path, low_memory=False)
x_test = pd.read_csv(x_test_file_path, low_memory=False)
c_x_test = pd.read_csv(cx_text, low_memory=False)
sample_set = pd.read_csv(sample_sub)

In [4]:
x_train = x_train.dropna(axis=1, how='all')
x_train = x_train.drop('cellattributevalue', axis=1)
x_train = x_train.drop('celllabel', axis=1)
x_train = x_train.drop('nursingchartcelltypevalname', axis=1)
x_train = x_train.drop('nursingchartvalue', axis=1)
x_train = x_train.drop('labmeasurenamesystem', axis=1)
x_train = x_train.drop('labname', axis=1)
x_train = x_train.drop('labresult', axis=1)
x_train = x_train.dropna()

In [5]:
x_test

,Unnamed: 0,admissionheight,admissionweight,age,cellattributevalue,celllabel,ethnicity,gender,labmeasurenamesystem,labname,labresult,nursingchartcelltypevalname,nursingchartvalue,offset,patientunitstayid,unitvisitnumber
0,0,149.9,NaN,59,NaN,NaN,Caucasian,Female,NaN,NaN,NaN,NaN,NaN,0.0,151179.0,1.0
1,1,165.1,86.8,66,NaN,NaN,Caucasian,Female,NaN,NaN,NaN,NaN,NaN,0.0,151900.0,1.0
2,2,170.2,81.0,41,NaN,NaN,Caucasian,Female,NaN,NaN,NaN,NaN,NaN,0.0,152954.0,1.0
3,3,193.0,163.8,39,NaN,NaN,Caucasian,Male,NaN,NaN,NaN,NaN,NaN,0.0,158056.0,1.0
4,4,162.6,49.2,20,NaN,NaN,Caucasian,Female,NaN,NaN,NaN,NaN,NaN,0.0,159411.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225890,225890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg/dL,glucose,159.0,NaN,NaN,2067.0,496980.0,NaN
225891,225891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg/dL,glucose,127.0,NaN,NaN,3402.0,496980.0,NaN
225892,225892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg/dL,glucose,155.0,NaN,NaN,-288.0,496980.0,NaN
225893,225893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg/dL,glucose,86.0,NaN,NaN,-85.0,2420527.0,NaN


In [6]:
x_test = x_test.dropna(axis=1, how='all')
x_test = x_test.drop('cellattributevalue', axis=1)
x_test = x_test.drop('celllabel', axis=1)
x_test = x_test.drop('nursingchartcelltypevalname', axis=1)
x_test = x_test.drop('nursingchartvalue', axis=1)
x_test = x_test.drop('labmeasurenamesystem', axis=1)
x_test = x_test.drop('labname', axis=1)
x_test = x_test.drop('labresult', axis=1)
x_test = x_test.drop('Unnamed: 0', axis=1)
x_test = x_test.dropna()

In [7]:
x_test

,admissionheight,admissionweight,age,ethnicity,gender,offset,patientunitstayid,unitvisitnumber
1,165.1,86.80,66,Caucasian,Female,0.0,151900.0,1.0
2,170.2,81.00,41,Caucasian,Female,0.0,152954.0,1.0
3,193.0,163.80,39,Caucasian,Male,0.0,158056.0,1.0
4,162.6,49.20,20,Caucasian,Female,0.0,159411.0,1.0
6,180.3,104.30,83,Caucasian,Male,0.0,162502.0,1.0
...,...,...,...,...,...,...,...,...
499,185.4,140.60,47,African American,Male,0.0,3343295.0,1.0
500,165.1,129.80,63,Caucasian,Female,0.0,3344944.0,2.0
501,172.7,75.00,73,Caucasian,Male,0.0,3347502.0,1.0
502,160.0,49.44,87,Caucasian,Female,0.0,3348293.0,2.0


In [8]:
merged_df = pd.merge(x_train, y_train, on='patientunitstayid', how='inner')
merged_df = merged_df.drop('Unnamed: 0_x', axis=1)
merged_df = merged_df.drop('Unnamed: 0_y', axis=1)
merged_df['age'] = pd.to_numeric(merged_df['age'], errors='coerce')
merged_df.dropna(inplace=True)

In [9]:
df1 = pd.get_dummies(merged_df['ethnicity'])
merged_df = pd.concat([merged_df, df1], axis=1).reindex(merged_df.index)
merged_df.drop('ethnicity', axis=1, inplace=True)
df2 = pd.get_dummies(merged_df['gender'])
merged_df = pd.concat([merged_df, df2], axis=1).reindex(merged_df.index)
merged_df.drop('gender', axis=1, inplace=True)

In [10]:
y_train = merged_df.pop('hospitaldischargestatus')

In [11]:
# loop through each column in the DataFrame
for col in merged_df.columns:
    # check if the column data type is float64
    if merged_df[col].dtype == 'float64':
        # convert the column to float32
        merged_df[col] = merged_df[col].astype('float32')

In [12]:
x_test['age'] = pd.to_numeric(x_test['age'], errors='coerce')
x_test.dropna(inplace=True)

    
# endoced dummies
df4 = pd.get_dummies(x_test['ethnicity'])
x_test = pd.concat([x_test, df4], axis=1).reindex(x_test.index)
x_test.drop('ethnicity', axis=1, inplace=True)

df3 = pd.get_dummies(x_test['gender'])
x_test = pd.concat([x_test, df3], axis=1).reindex(x_test.index)
x_test.drop('gender', axis=1, inplace=True)


# fix merged_df
merged_df = merged_df.drop('Native American', axis=1)

In [13]:
c_x_test = c_x_test.drop('cellattributevalue', axis=1)
c_x_test = c_x_test.drop('celllabel', axis=1)
c_x_test = c_x_test.drop('nursingchartcelltypevalname', axis=1)
c_x_test = c_x_test.drop('nursingchartvalue', axis=1)
c_x_test = c_x_test.drop('labmeasurenamesystem', axis=1)
c_x_test = c_x_test.drop('labname', axis=1)
c_x_test = c_x_test.drop('labresult', axis=1)
c_x_test = c_x_test.drop('Unnamed: 0', axis=1)

# endoced dummies
df4 = pd.get_dummies(c_x_test['ethnicity'])
c_x_test = pd.concat([c_x_test, df4], axis=1).reindex(c_x_test.index)
c_x_test.drop('ethnicity', axis=1, inplace=True)

df3 = pd.get_dummies(c_x_test['gender'])
c_x_test = pd.concat([c_x_test, df3], axis=1).reindex(c_x_test.index)
c_x_test.drop('gender', axis=1, inplace=True)

c_x_test = c_x_test.fillna(0)

c_x_test['age'] = pd.to_numeric(c_x_test['age'], errors='coerce').fillna(90).astype('float32')

# loop through each column in the DataFrame
for col in c_x_test.columns:
    # check if the column data type is float64
    if c_x_test[col].dtype == 'float64':
        # convert the column to float32
        c_x_test[col] = c_x_test[col].astype('float32')
        
c_x_test

,admissionheight,admissionweight,age,offset,patientunitstayid,unitvisitnumber,African American,Asian,Caucasian,Hispanic,Other/Unknown,Female,Male
0,149.899994,0.000000,59.0,0,151179,1,0,0,1,0,0,1,0
1,165.100006,86.800003,66.0,0,151900,1,0,0,1,0,0,1,0
2,170.199997,81.000000,41.0,0,152954,1,0,0,1,0,0,1,0
3,193.000000,163.800003,39.0,0,158056,1,0,0,1,0,0,0,1
4,162.600006,49.200001,20.0,0,159411,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,185.399994,140.600006,47.0,0,3343295,1,1,0,0,0,0,0,1
500,165.100006,129.800003,63.0,0,3344944,2,0,0,1,0,0,1,0
501,172.699997,75.000000,73.0,0,3347502,1,0,0,1,0,0,0,1
502,160.000000,49.439999,87.0,0,3348293,2,0,0,1,0,0,1,0


In [14]:
# Create SVM classifier
classifier = svm.SVC(kernel='sigmoid', C=0.1, gamma='auto')

# Train the model
classifier.fit(merged_df, y_train)

# Make predictions
y_pred = classifier.predict(merged_df)

# Evaluate the model
accuracy = classifier.score(merged_df, y_train)
print('Accuracy:', accuracy)

Accuracy: 0.925476603119584


In [15]:
y_pred = classifier.predict(c_x_test)

In [16]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [23]:
#model = Sequential()
#model.add(SimpleRNN(units=32, input_shape=(merged_df.shape[1], 1)))
#model.add(Dense(units=1, activation='sigmoid'))

model = Sequential()
model.add(Dense(32, input_dim = merged_df.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))


# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(merged_df, y_train, epochs = 10, batch_size=32, verbose=1)

mse = model.evaluate(merged_df, y_train)
print("Mean Squared Error: ", mse)

# Use moel to predict labels of new data
y_pred = model.predict(c_x_test)

# Convert predicted labels back to dataframe format
df_pred = pd.DataFrame(y_pred, columns=['hospitaldischargestatus'])

Epoch 1/50
55/55 [==============================] - 1s 2ms/step - loss: 30232971264.0000
Epoch 2/50
55/55 [==============================] - 0s 2ms/step - loss: 111225920.0000
Epoch 3/50
55/55 [==============================] - 0s 2ms/step - loss: 298199.0938
Epoch 4/50
55/55 [==============================] - 0s 2ms/step - loss: 428.2967
Epoch 5/50
55/55 [==============================] - 0s 2ms/step - loss: 108.1379
Epoch 6/50
55/55 [==============================] - 0s 2ms/step - loss: 105.1698
Epoch 7/50
55/55 [==============================] - 0s 2ms/step - loss: 105.1580
Epoch 8/50
55/55 [==============================] - 0s 2ms/step - loss: 105.1767
Epoch 9/50
55/55 [==============================] - 0s 2ms/step - loss: 105.2463
Epoch 10/50
55/55 [==============================] - 0s 2ms/step - loss: 105.1719
Epoch 11/50
55/55 [==============================] - 0s 2ms/step - loss: 105.0129
Epoch 12/50
55/55 [==============================] - 0s 2ms/step - loss: 105.4636
Epoch 13

In [24]:
merged_df.shape[1]

13

In [25]:
# Extract one column from the first dataframe
col = c_x_test['patientunitstayid']
#col
#df_pred
# Concatenate the column with the second dataframe
result = pd.concat([col, df_pred], axis=1)
result = result.dropna()
result

,patientunitstayid,hospitaldischargestatus
0,151179,20.043962
1,151900,17.968767
2,152954,15.574237
3,158056,13.039081
4,159411,13.778338
...,...,...
499,3343295,-12.207990
500,3344944,-11.411115
501,3347502,-6.895490
502,3348293,-4.817365


In [20]:
result.to_csv('result.csv', index=False)